We thank Sagi Haim for developing this script

### Black & Scholes Option Pricing

In [ ]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
from scipy.optimize import minimize
import datetime

Black-Scholes, or sometimes Black-Scholes-Merton, is a mathematical model that seeks to explain the behavior of financial derivatives, most commonly options. It was proposed by Black and Scholes in 1973. It gave theoretical support for trading options to hedge positions, which had been practice but lacked solid support. From the model we are able to calculate what the price of an option should be based on a number of different factors. Nowadays there are numerous variations of the Black-Scholes model, each of which seeks to improve the model based on certain criteria, usually at the cost of a significant increase in complexity. This paper will focus on the original model, the basis for all other models.

There is a bit of notation, we're going to lay it down before we get to the equations.

-   C = Call option price

-   S = Current stock price

-   X = Strike price of the option

-   r = risk-free interest rate (a number between 0 and 1)

-   $\sigma$ = volatility of the stocks return (a number between 0 and 1)

-   t = time to option maturity (in years)

-   N = normal cumulative distribution function

**The model:**

$$\mathrm C(\mathrm S,\mathrm t)= \mathrm N(\mathrm d_1)\mathrm S - \mathrm N(\mathrm d_2) \mathrm X \mathrm e^{-rt}$$

Where:

$$C_0 = \text{the value of a European option at time t = 0.}$$

$$\mathrm d_1= \frac{1}{\sigma \sqrt{\mathrm t}} \left[\ln{\left(\frac{S}{X}\right)} + t\left(r + \frac{\sigma^2}{2} \right) \right]$$

$$\mathrm d_2= d_1-\sigma\sqrt{t}$$
$$N(x)=\frac{1}{\sqrt{2\pi}} \int_{-\infty}^{x} \mathrm e^{-\frac{1}{2}z^2} dz$$


$N(d)$ = Probability that a standardized, normally distributed, random variable will be less than or equal to $d$.

------------------------------------------------------------------------

Find the value of a six-month call option on Hardcraft, Inc. with an exercise price of $150. The current value of a share of Hardcraft is $160. The interest rate available in the U.S. is R = 5%. The option maturity is 6 months (half of a year). The volatility of the underlying asset is 30% per annum. Note that the intrinsic value of the option is $10, so our answer must be at least that amount.


In [ ]:
S, X, r, t, sigma_S = 160, 150, 0.05, 0.5, 0.3
#S, X, r, t, sigma_S = 50, 45, 0.04, 0.75, 0.3

**Function: d1**

In [ ]:
def bs_d1(S, X, t, r, sigma_S, k=0):
    return (math.log(S/X) + (r - k + 0.5 * sigma_S ** 2) * t) / (sigma_S * math.sqrt(t))

In [ ]:
d1 = bs_d1(S, X, t, r, sigma_S)
print("d1",d1)

**Function: d2**

In [ ]:
def bs_d2(S, X, t, r, sigma_S, k=0):
    return (math.log(S/X) + (r - k + 0.5 * sigma_S ** 2) * t) / (sigma_S * math.sqrt(t)) - sigma_S * math.sqrt(t)

In [ ]:
d2 = bs_d2(S, X, t, r, sigma_S)
print("d2",d2)

**Function: N(d1)**

In [ ]:
def bs_Nd1(S, X, t, r, sigma_S, k=0):
    d1 = (math.log(S/X) + (r - k + 0.5 * sigma_S ** 2) * t) / (sigma_S * math.sqrt(t))
    return norm.cdf(d1)

In [ ]:
Nd1 = bs_Nd1(S, X, t, r, sigma_S)
print("N(d1)",Nd1)

**Function: N(d2)**

In [ ]:
def bs_Nd2(S, X, t, r, sigma_S, k=0):
    d2 = (math.log(S/X) + (r - k + 0.5 * sigma_S ** 2) * t) / \
        (sigma_S * math.sqrt(t)) - sigma_S * math.sqrt(t)
    return norm.cdf(d2)

In [ ]:
Nd2 = bs_Nd2(S, X, t, r, sigma_S)
print("N(d2)",Nd2)

**Funciton: BS Call option**

In [ ]:
def bs_call(S, X, t, r, sigma_S, k=0):
    d1 = (math.log(S/X) + (r - k + 0.5 * sigma_S ** 2) * t) / (sigma_S * math.sqrt(t))
    d2 = d1 - sigma_S * math.sqrt(t)
    return S * norm.cdf(d1) * math.exp(-k * t) - X * math.exp(-r*t) * norm.cdf(d2)

In [ ]:
call = bs_call(S, X, t, r, sigma_S)
print("Call",call)

**Funciton: BS Put option**

In [ ]:
def bs_put(S, X, t, r, sigma_S, k=0):
    d1 = (math.log(S/X) + (r - k + 0.5 * sigma_S ** 2) * t) / (sigma_S * math.sqrt(t))
    d2 = d1 - sigma_S * math.sqrt(t)
    return - S * norm.cdf(-d1) * math.exp(-k * t) + X * math.exp(-r*t) * norm.cdf(-d2)

In [ ]:
put = bs_put(S, X, t, r, sigma_S)
print("Put",put)

## Implied Volatility

In [ ]:
S = 51
X = 50
r = 0.05
t = 1.25
target = 6.0

In [ ]:
# Function: Implied Volatility
def bs_iv(S, X, t, r, target, k=0, type="C"):
    high = 2
    low = 0

    # Funciton: BS Call option
    def bs_call(S, X, t, r, sigma_S, k=0):
        d1 = (math.log(S/X) + (r - k + 0.5 * sigma_S ** 2) * t) / (sigma_S * math.sqrt(t))
        d2 = d1 - sigma_S * math.sqrt(t)
        return S * norm.cdf(d1) * math.exp(-k * t) - X * math.exp(-r*t) * norm.cdf(d2)

    # Funciton: BS Put option

    def bs_put(S, X, t, r, sigma_S, k=0):
        d1 = (math.log(S/X) + (r - k + 0.5 * sigma_S ** 2) * t) / (sigma_S * math.sqrt(t))
        d2 = d1 - sigma_S * math.sqrt(t)
        return - S * norm.cdf(-d1) * math.exp(-k * t) + X * math.exp(-r*t) * norm.cdf(-d2)

    if type == "C":
        while (high - low) > 0.0001:
            if bs_call(S, X, t, r, (high + low)/2, k) - target > 0:
                high = (high + low) / 2
            elif bs_call(S, X, t, r, (high + low)/2, k) - target < 0:
                low = (high + low) / 2
                pass
            pass
        return (high + low) / 2
        pass

    if type == "P":
        while (high - low) > 0.0001:
            if bs_put(S, X, t, r, (high + low)/2, k) - target > 0:
                high = (high + low) / 2
            elif bs_put(S, X, t, r, (high + low)/2, k) - target < 0:
                low = (high + low) / 2
                pass
            pass
        return (high + low) / 2
        pass

In [ ]:
iv_call = bs_iv(S, X, t, r, target, type="C")
print("Implied Volatility Call",iv_call)

In [ ]:
iv_put = bs_iv(S, X, t, r, target, type="P")
print("Implied Volatility Put",iv_put)

## Dividend Adjustments

**Dividend Adjustments to the Black-Scholes - A Known Dividend**

In [ ]:
# Input
today = datetime.datetime(2019, 8, 28)
expiration = datetime.datetime(2020, 2, 21)
S0 = 54.99
r = 0.0173

In [ ]:
# Expected Dividends
div_1_date = datetime.datetime(2019, 9, 12)
div_2_date = datetime.datetime(2019, 11, 29)
div_1_payment = 0.4
div_2_payment = 0.4

In [ ]:
# Defining t
delta_t_1 = (div_1_date - today).days / 365
delta_t_2 = (div_2_date - today).days / 365

In [ ]:
# finding PV(Dividends)
pv_div_1 = div_1_payment * math.exp(-r * delta_t_1)
pv_div_2 = div_2_payment * math.exp(-r * delta_t_2)

In [ ]:
# Calculating net S0
net_S0 = S0 - pv_div_1 - pv_div_2
print("Net S0",net_S0)

In [ ]:
# Calculating Black and Scholes Implies Volatility
X = 55
ex_date = datetime.datetime(2020, 2, 21)
t = (ex_date - today).days / 365
call_price = 2.5
put_price = 3.25

In [ ]:
# Implied Volatility, net of dividends - Call
iv_net_call = bs_iv(S=net_S0, X=X, t=t, r=r, target=call_price, type="C")
print("Implied Volatility, net of dividends - Call",iv_net_call)

In [ ]:
# Implied Volatility, net of dividends - Put
iv_net_put = bs_iv(S=net_S0, X=X, t=t, r=r, target=put_price, type="P")
print("Implied Volatility, net of dividends - Put",iv_net_put)

In [ ]:
# Implied Volatility, with dividends - Call
iv_call = bs_iv(S=S0, X=X, t=t, r=r, target=call_price, type="C")
print("Implied Volatility, with dividends - Call",iv_call)

In [ ]:
# Implied Volatility, with dividends - Put
iv_put = bs_iv(S=S0, X=X, t=t, r=r, target=put_price, type="P")
print("Implied Volatility, with dividends - Put",iv_put)

### Dividend Adjustments for Continuous Dividend Payouts-The Merton Model

In [ ]:
# Price SP500 Spiders
# Input
S0 = 292.45
X = 300
t = 0.296
r = 0.0195
k = 0.017  # Dividend rate
sigma_S = 0.14

In [ ]:
# Implied Volatility, net of continuous dividends - Call
iv_call = bs_iv(S=S0, X=X, t=t, r=r, target=call_price, k=k, type="C")
print("Implied Volatility, net of continuous dividends - Call",iv_call)

In [ ]:
# Implied Volatility, net of continuous dividends - Put
iv_put = bs_iv(S=S0, X=X, t=t, r=r, target=call_price, k=k, type="P")
print("Implied Volatility, net of continuous dividends - Put",iv_put)

In [ ]:
# d1
print("d1",bs_d1(S0, X, t, r, sigma_S, k))

In [ ]:
# d2
print("d2",bs_d2(S0, X, t, r, sigma_S, k))

In [ ]:
# Nd1
print("N(d1)",bs_Nd1(S0, X, t, r, sigma_S, k))

In [ ]:
# Nd2
print("N(d2)",bs_Nd2(S0, X, t, r, sigma_S, k))

In [ ]:
# BS Call
print("BS Call",bs_call(S0, X, t, r, sigma_S, k))

In [ ]:
# BS Put
print("BS Put",bs_put(S0, X, t, r, sigma_S, k))

In [ ]:
# Pricing an Option to buy Australian Dollars in US Dollars
# Input
S0 = 0.6718
X = 0.7
rUS = 0.0195
rAUD = 0.0093
t = 0.2959
sigma_S = 0.0970

In [ ]:
# d1
print("d1",bs_d1(S0, X, t, rUS, sigma_S, k=rAUD))

In [ ]:
# d2
print("d2",bs_d2(S0, X, t, rUS, sigma_S, k=rAUD))

In [ ]:
# Nd1
print("N(d1)",bs_Nd1(S0, X, t, rUS, sigma_S, k=rAUD))

In [ ]:
# Nd2
print("N(d2)",bs_Nd2(S0, X, t, rUS, sigma_S, k=rAUD))

In [ ]:
# BS Call
print("BS Call",bs_call(S0, X, t, rUS, sigma_S, k=rAUD) * 10000)

In [ ]:
# BS Put
print("BS Put",bs_put(S0, X, t, rUS, sigma_S, k=rAUD) * 10000)

### The Black (1976) Model for Bond Option Valuation

In [ ]:
# Input
Fwd = 133  # Bond Forward price
X = 130  # Exercise price
r = 0.02
t = 0.5
sigma_S = 0.06

In [ ]:
# Call 
d1 = bs_d1(Fwd, X, t, 0, sigma_S)
d2 = bs_d2(Fwd, X, t, 0, sigma_S)
Nd1 = bs_Nd1(Fwd, X, t, 0, sigma_S)
Nd2 = bs_Nd2(Fwd, X, t, 0, sigma_S)
call_price = math.exp(-r * t) * (Fwd * Nd1 - X * Nd2)
print("Call",call_price)

In [ ]:
# Put
N_minus_d1 = norm.cdf(-d1)
N_minus_d2 = norm.cdf(-d2)
put_price = math.exp(-r * t) * (X * N_minus_d2 - Fwd * N_minus_d1)
print("Put",put_price)

### Using the Black-Scholes Model to Price Risky Debt

In [ ]:
# Input
V = 1400  # firm Asset value today
FVt = 1150  # debt payment at maturity
r = 0.02  # risk free rate
t = 2.5  # time to debt maturity
sigma_S = 0.2  # assets volatility

In [ ]:
# Equity Value
d1 = bs_d1(V, FVt, t, 0, sigma_S)
d2 = bs_d2(V, FVt, t, 0, sigma_S)
Nd1 = bs_Nd1(V, FVt, t, 0, sigma_S)
Nd2 = bs_Nd2(V, FVt, t, 0, sigma_S)
equity_value = V * Nd1 - FVt * math.exp(-r * t) * Nd2
print("Equity Value",equity_value)

In [ ]:
# Debt VAlue and 
debt_value = V - equity_value
print("Debt Value",debt_value)

In [ ]:
# Yield to maturity
YTM = (FVt / debt_value)**(1 / t) - 1
print("YTM",YTM)

In [ ]:
# The KMV model - Probability to Default
default_prob = 1 - norm.cdf(d2)
print("The KMV model - Probability to Default",default_prob)